<a href="https://colab.research.google.com/github/AdiKaipa/END/blob/main/Sentiment_Analysis_on_Stanford_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from google.colab import drive 
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [95]:
import pandas as pd
import numpy as np


In [96]:
data_path = '/content/drive/MyDrive/END/datasetSentences.txt'
label_path = '/content/drive/MyDrive/END/sentiment_labels.txt'
dict_path = '/content/drive/MyDrive/END/dictionary.txt'

In [97]:
data = pd.read_csv(data_path, sep = '\t' )
label = pd.read_csv(label_path, sep = '|')
dictionary = pd.read_csv(dict_path, sep = '|', header=None)
print(data.shape)
print(label.shape)
print(dictionary.shape)

(11855, 2)
(239232, 2)
(239232, 2)


In [98]:
dictionary.columns = ["phrase", "id"]
label.columns = ["id", "sentiment"]

In [99]:
data.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [100]:
sentences = data['sentence']
labels = label[1: data.shape[0]+ 1]['sentiment']
len(sentences), len(labels)

                      

(11855, 11855)

In [101]:
!pip install googletrans==3.1.0a0

In [102]:

import random
import googletrans
from googletrans import Translator
def Back_Translate(sentence):
  translator = Translator()
  #sentence = [sentence]
  available_langs = list(googletrans.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")
  translations = translator.translate(sentence, dest= trans_lang)
  t_text = [t.text for t in translations]
  print(t_text)

  translations_en_random = translator.translate(t_text, src=trans_lang, dest='en')
  en_text = [t.text for t in translations_en_random]
  return en_text
  print(en_text)

In [103]:
def random_swap(sentence, n=5): 
    sentence = sentence.split()
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return ' '.join(sentence)

In [104]:
import random
def random_deletion(words, p=0.5): 
    words = words.split()
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,len(words)) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [105]:
#data_translated = Back_Translate(data['sentence'].tolist())

In [106]:
swap_data =[]
for s in sentences:
  swap_data.append(random_swap(s))

In [107]:
data_swap_random = []
for s in swap_data:
  delet = random_deletion(s)

  data_swap_random.append(' '.join(delet))

In [108]:
sent_label = []
for x in labels:
  if x <= 0.2:
    x = 0
  elif x<= 0.4:
    x = 1
  elif x <=0.6:
    x = 2
  elif x <= 0.8:
    x = 3
  elif x <= 1.0:
    x = 4
  sent_label.append(x)

In [109]:
df = pd.DataFrame(list(zip(data_swap_random, sent_label)), columns = ['tweets', 'labels'])
df.shape

(11855, 2)

In [110]:
df.head()

,tweets,labels
0,The or going destined to be even 21st Century ...,2
1,The gorgeously elaborate '' of `` Lord The of ...,2
2,biopic but too-tepid Effective,2
3,"If , you like to to to the movies to have fun ...",2
4,"Emerges n't that as , an issue movie something...",1


In [111]:
import random 
import torch, torchtext 
from torchtext import data

In [112]:
SEED = 43
torch.manual_seed(SEED)

In [114]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first = True,include_lengths = True)
Label = data.LabelField(tokenize = 'spacy', is_target = True, batch_first = True, sequential = False)

In [115]:
fields = [('tweets', Tweet), ('labels', Label)]

In [116]:
example = [data.Example.fromlist([df.tweets[i], df.labels[i]], fields) for i in range(df.shape[0])]

In [117]:
twitterDataset = data.Dataset(example, fields)

In [118]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state = random.seed(SEED))

In [119]:
len(train), len(valid)

(10077, 1778)

In [120]:
vars(train.examples[0])

{'labels': 3,
 'tweets': ['nothing',
  'swashbuckling',
  '.',
  'like',
  ',',
  'Sometimes',
  'satisfies']}

In [121]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [122]:
print("Size of input vocab", len(Tweet.vocab))
print("Size of label vocab", len(Label.vocab))
print("Top 10 words appeared repeatdly", list(Tweet.vocab.freqs.most_common(10)))
print("Label: ", Label.vocab.stoi)

Size of input vocab 18586
Size of label vocab 5
Top 10 words appeared repeatdly [('.', 9194), (',', 8260), ('the', 7006), ('and', 5117), ('a', 5061), ('of', 5040), ('to', 3510), ('-', 3146), ("'s", 2926), ('is', 2899)]
Label:  defaultdict(<function _default_unk_index at 0x7f12617ecae8>, {2: 0, 1: 1, 3: 2, 4: 3, 0: 4})


In [123]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [124]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                     sort_key = lambda x: len(x.tweets),
                                                     sort_within_batch = True, device = device)

In [125]:
import os, pickle 
with open('tokenizer.pkl', 'wb') as tokens:
  pickle.dump(Tweet.vocab.stoi, tokens)

In [126]:
import torch.nn as nn
import torch.nn.functional as F

In [127]:
class Classifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
    super().__init__()
    #Embedding Layer

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    #LSTM layer
    self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers = n_layers, 
                           dropout = dropout, 
                           batch_first = True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text, text_lengths):
    embedded = self.embedding(text)
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first = True)
    packed_output, (hidden, cell) = self.encoder(packed_embedded)
    dense_outputs = self.fc(hidden)
    output = F.softmax(dense_outputs[0], dim = 1)
    return output

In [135]:
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2
model = Classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout)

In [136]:
print(model)

Classifier(
  (embedding): Embedding(18586, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)


In [137]:
#no. of trainable parameters
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'model has {count_parameters(model):,} trainable parametrs' )

model has 5,817,905 trainable parametrs


In [138]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr = 2e-4)
criterion = nn.CrossEntropyLoss()

def binary_accuracy(preds, y):
  _, predictions = torch.max(preds, 1)
  correct = (predictions == y).float()
  acc = correct.sum() / len(correct)
  return acc

model = model.to(device)
criterion = criterion.to(device)

In [139]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.train()
  for batch in iterator:
    optimizer.zero_grad()
    tweet, tweet_lengths = batch.tweets 
    predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
    loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
    acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
    loss.backward()       
        
        # update the weights
    optimizer.step()      
        
        # loss and accuracy
    epoch_loss += loss.item()  
    epoch_acc += acc.item()    
        
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [140]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [141]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.426 | Train Acc: 51.51%
	 Val. Loss: 1.386 |  Val. Acc: 52.08% 

	Train Loss: 1.379 | Train Acc: 52.68%
	 Val. Loss: 1.385 |  Val. Acc: 52.08% 

	Train Loss: 1.378 | Train Acc: 52.69%
	 Val. Loss: 1.385 |  Val. Acc: 52.08% 

	Train Loss: 1.378 | Train Acc: 52.72%
	 Val. Loss: 1.385 |  Val. Acc: 52.08% 

	Train Loss: 1.377 | Train Acc: 52.78%
	 Val. Loss: 1.385 |  Val. Acc: 52.08% 

	Train Loss: 1.376 | Train Acc: 52.96%
	 Val. Loss: 1.385 |  Val. Acc: 52.02% 

	Train Loss: 1.375 | Train Acc: 53.29%
	 Val. Loss: 1.385 |  Val. Acc: 51.97% 

	Train Loss: 1.373 | Train Acc: 53.49%
	 Val. Loss: 1.385 |  Val. Acc: 51.97% 

	Train Loss: 1.371 | Train Acc: 53.69%
	 Val. Loss: 1.385 |  Val. Acc: 51.97% 

	Train Loss: 1.369 | Train Acc: 53.97%
	 Val. Loss: 1.385 |  Val. Acc: 52.02% 

